In [32]:
#!pip install mysql.connector
import mysql.connector
import pandas as pd


In [33]:
#Conectar banco de dados MySQL
mydb = mysql.connector.connect(
    host="34.151.255.31",
    user="root",
    password="1234",
    database="banco_case_gb"
)

In [34]:
# Ler o Excel por meio do Pandas
df = pd.read_excel("./Arquivos/Base_2017.xlsx")

In [35]:
# Cursor para executar a consulta SQL
cursor = mydb.cursor()


In [36]:
# Definir a consulta SQL para verificar se o registro já existe na tabela
sql_check = "SELECT COUNT(*) FROM banco_case_gb.raw_vendas WHERE ID_MARCA=%s AND ID_LINHA=%s AND DATA_VENDA=%s AND QTD_VENDA=%s"

In [37]:
# Definir a consulta SQL para inserir dados na tabela
sql_insert = "INSERT INTO banco_case_gb.raw_vendas (ID_MARCA, MARCA, ID_LINHA, LINHA, DATA_VENDA, QTD_VENDA) VALUES (%s, %s, %s, %s, %s, %s)"

In [38]:
# Controle "DELTA" para Insert dos dados
# Percorrer as linhas do arquivo Excel e executar a consulta SQL com os valores correspondentes
for index, row in df.iterrows():
    # Tratar a coluna de data
    data_venda = pd.to_datetime(row['DATA_VENDA'], format='%d/%m/%Y').strftime('%Y-%m-%d')
    
    # Verificar se o registro já existe na tabela
    val_check = (row['ID_MARCA'], row['ID_LINHA'], data_venda, row['QTD_VENDA'])
    cursor.execute(sql_check, val_check)
    result = cursor.fetchone()
    
    # Se o registro não existe, executar a consulta SQL de inserção
    if result[0] == 0:
        val_insert = (row['ID_MARCA'], row['MARCA'], row['ID_LINHA'], row['LINHA'], data_venda, row['QTD_VENDA'])
        cursor.execute(sql_insert, val_insert)

In [39]:
# Confirma as alterações no Banco
mydb.commit()

In [40]:
print(cursor.rowcount, "linha(s) inserida(s).")

1 linha(s) inserida(s).


In [41]:
# Fechar a conexão com o banco de dados
cursor.close()
mydb.close()